# Multinomial Logistic Regression Recursive Feature Elimination

Brian Ward  
brian@brianpward.net  
https://github.com/etnite

This notebook is one attempt to create a method to identify a minimal set of predictors (SNPs) which fully describe a non-ordinal categorical response (haplotypes) for a given region of the genome. My idea is to perform a (possibly multinomial) logistic regression, using recursive feature elimination (RFE) to select a good subset of SNPs

There are many different methods for finding "tag SNPs" which can capture the variance explained by multiple surrounding SNPs. This problem is non-trivial and potentially NP-hard. See https://doi.org/10.1109/CSB.2005.22 for one example and some background on different methods. Note that much of this work was performed around the time of the first human HapMap project (ca. 2005), and hence there are many links to software on websites which no longer exist. These programs are also likely to be written in C or Perl, and getting them to run and use current data formats can be very difficult.

Many tag SNP identification algorithms are designed to find sets of tag SNPs across chromosomes/genomes, without any phenotypic data. I think that my particular use-case here is actually somewhat simpler, since I:

1. Have access to a "phenotype" (haplotype groups determined by a simple distance matrix using all SNPs/variants in the region of interest)
2. Only need to focus on a single region, obviating the need to find haploblock boundaries

This script performs repeated k-fold validation with the full set of SNPs in a region, with the selected subset of *n* SNPs identified by scikit-learn's RFE algorithm, and with the first *n* principal components of the full SNP matrix. We use the handy scikit-allel module to import data from a VCF file http://alimanfoo.github.io/2017/06/14/read-vcf.html

In [8]:
from sklearn.feature_selection import RFE
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, KFold
from sklearn import preprocessing
import allel
import numpy as np
import pandas as pd

## User-Defined Constants

Constants are as follows:

* Path to VCF (genotypic data) file
* Region of interest formatted chr:start_pos-end_pos
* .csv file of haplotypes - must contain at least two cols. - one for sample, one for the haplotype groupings
* response - string identifying the response variable in the haplotypes file
* n_snps - Desired number of SNPs to retain
* n_reps - Number of times to repeat k-fold validation
* val_k - Number of folds per repetition of k-fold validation

In [2]:
vcf_file = '/home/brian/repos/manuscripts/manu_2018_stripe_rust/input_data/geno/GAWN_Yr_postimp_filt_KASP.vcf.gz'
reg = '4B:526943215-598847646'
haps_file = '/home/brian/Downloads/haplotype_groupings.csv'
response = '4BL'
min_snps = 4
max_snps = 5
n_reps = 5
val_k = 5

## Read input

Here we read in the region of interest from the VCF file. We then convert the GT calls from the VCF into a genotype array (this is an intermediate step, and I'm currently not sure if it is required or not).

In [3]:
vcf = allel.read_vcf(vcf_file, region = reg)
preds = vcf['variants/ID']
gt = allel.GenotypeArray(vcf['calldata/GT'])
print("Number of Samples:", len(vcf['samples']))
print("Number of SNPs:", len(preds))
gt

FileNotFoundError: [Errno 2] No such file or directory: '/mnt/DATAPART2/brian/repos/manuscripts/manu_2020_Yr/input_data/geno/GAWN_KM_Yr_postimp_filt.vcf.gz'

Now we convert the genotype array into minor allele dosage format, then convert to a dataframe, set SNP IDs as col. names, and add a column for the sample name.

In [4]:
dos = gt.to_n_alt()
dos = dos.transpose()

dos_df = pd.DataFrame(dos)
dos_df.columns = vcf["variants/ID"]
dos_df["sample"] = vcf["samples"]

Now import the haplotypes file, merge it with the dos_df dataframe, and set the response vector (y) and predictors matrix (X). We then standardize each predictor to form the X_std matrix.

In [5]:
haps = pd.read_csv(haps_file)
haps = haps[["sample", response]]

merged = pd.merge(haps, dos_df, how = "inner", on = "sample")
y = merged["4BL"]
X = merged[vcf["variants/ID"]]


sc = preprocessing.StandardScaler()
X_std = sc.fit_transform(X)
X_std

array([[-1.14019606, -0.71251567, -0.33202381, ...,  1.00469512,
         1.01349415,  1.36287906],
       [ 0.92052526, -0.71251567, -0.33202381, ..., -1.01701007,
        -1.01555409, -0.74952987],
       [ 0.92052526, -0.71251567, -0.33202381, ..., -1.01701007,
        -1.01555409, -0.74952987],
       ...,
       [-1.14019606, -0.71251567, -0.33202381, ...,  1.00469512,
         1.01349415,  1.36287906],
       [ 0.92052526, -0.71251567, -0.33202381, ...,  1.00469512,
         1.01349415, -0.74952987],
       [ 0.92052526,  1.45361993, -0.33202381, ..., -1.01701007,
        -1.01555409, -0.74952987]])

## Initialize Output Structures

Here we need to create a list of dataframes to store the output statistics.

In [5]:
proto_df = 'a'

## Logistic Regression Feature Elimination

Here we create a multinomial logistic regressor. Then we utilize recursive feature elimination (RFE) to try and remove non-informative/repetitive features (i.e. SNPs) until we get down to the number of SNPs chosen by the user.

In [6]:
mlogit = LogisticRegression(solver = 'lbfgs', penalty = 'none', multi_class = 'auto', max_iter = 5e3)
rfe = RFE(mlogit, n_snps)
fit = rfe.fit(X_std, y)

In [7]:
sub_preds = preds[fit.support_]
print("Retained features:", sub_preds)
X_sub = X_std[:, fit.support_]

print("Retained features array first 5 rows:")
X_sub[:5]

Retained features: ['S4B_546015349' 'S4B_546664811' 'S4B_592005439' 'S4B_596498722']
Retained features array first 5 rows:


array([[-0.97658556, -0.979656  , -0.82095677, -0.84629444],
       [ 1.05067846,  1.05398185, -0.82095677, -0.84629444],
       [ 1.05067846,  1.05398185, -0.82095677, -0.84629444],
       [ 1.05067846,  1.05398185,  1.24718241,  1.20690759],
       [-0.97658556, -0.979656  ,  1.24718241,  1.20690759]])

### Principal Component Analysis

We can also perform logistic regression using the same number of principal components as the number of SNPs we decided to use. This should give us a theoretical maximum predicted vs. observed haplotypes accuracy for a given number n of predictors.

In [8]:
pca = PCA(n_components = n_snps)
pca.fit(np.transpose(X_std))
X_pcs = pca.components_.T

'''
Another variation - find the SNPs most highly correlated with each PC
loadings = np.abs(pca.components_)
top_corrs = np.argmax(loadings, axis = 1)
X_pcs = X_std[:, top_corrs]
'''

'\nAnother variation - find the SNPs most highly correlated with each PC\nloadings = np.abs(pca.components_)\ntop_corrs = np.argmax(loadings, axis = 1)\nX_pcs = X_std[:, top_corrs]\n'

## K-Fold Cross-validation

Now we perform n repeats of k-fold cross-validation, and find the mean accuracy of the prediction (proportion of times the predicted response category matches the observed category). We do this for the full set of SNPs, the chosen subset of *n* SNPs, and the first *n* principal components.

In [9]:
full_score_arr = np.zeros((n_reps, val_k), dtype = np.float)
for i in range(0, n_reps):
    k_fold = KFold(n_splits = val_k, random_state = i, shuffle = True)
    full_score_arr[i,:] = cross_val_score(mlogit, X_std, y, cv = k_fold, scoring = 'accuracy')

sub_score_arr = np.zeros((n_reps, val_k), dtype = np.float)
for i in range(0, n_reps):
    k_fold = KFold(n_splits = val_k, random_state = i, shuffle = True)
    sub_score_arr[i,:] = cross_val_score(mlogit, X_sub, y, cv = k_fold, scoring = 'accuracy')

pc_score_arr = np.zeros((n_reps, val_k), dtype = np.float)
for i in range(0, n_reps):
    k_fold = KFold(n_splits = val_k, random_state = i, shuffle = True)
    pc_score_arr[i,:] = cross_val_score(mlogit, X_pcs, y, cv = k_fold, scoring = 'accuracy')

In [10]:
print("Prediction accuracy, all SNPs:")
print(round(np.mean(full_score_arr), 2))

print("\nPrediction accuracy, selected SNPs:")
print(round(np.mean(sub_score_arr), 2))

print("\nPrediction accuracy, PCs:")
print(round(np.mean(pc_score_arr), 2))

Prediction accuracy, all SNPs:
0.98

Prediction accuracy, selected SNPs:
0.85

Prediction accuracy, PCs:
0.99


## Conclusion

In this case, it appears that there is no small subset of SNPs that have very a high predictive accuracy when compared against the ~100% prediction accuracy using all 427 SNPs used to identify the haplotypes in the first place. Notably, using just the first handful of principal components of the genotypic matrix, we can almost perfectly predict the haplotypes. However, an alternative strategy, which was to use the SNPs most highly correlated with each PC, yielded similar results the RFE method (mean accuracy of ~ 0.85, method not shown here)